# Data Preparation Notebook

In this notebook, you will execute code to 

1. download [MovieLens](https://grouplens.org/datasets/movielens/) dataset into `ml-latest-small` directory
2. split the data into training and testing sets
3. perform negative sampling
4. calculate statistics needed to train the NCF model
5. upload data onto S3 bucket

## 1. Download dataset

In [7]:
%%bash
# delete the data directory if exists
rm -r ../../data/recommendations/ml-latest-small

# download movielens small dataset
curl -O http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

# unzip into data directory
DATA_DIR="../../data/recommendations/"
unzip ml-latest-small.zip -d $DATA_DIR
rm ml-latest-small.zip

rm: cannot remove '../../data/recommendations/ml-latest-small': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   589k      0  0:00:01  0:00:01 --:--:--  589k


Archive:  ml-latest-small.zip
   creating: ../../data/recommendations/ml-latest-small/
  inflating: ../../data/recommendations/ml-latest-small/links.csv  
  inflating: ../../data/recommendations/ml-latest-small/tags.csv  
  inflating: ../../data/recommendations/ml-latest-small/ratings.csv  
  inflating: ../../data/recommendations/ml-latest-small/README.txt  
  inflating: ../../data/recommendations/ml-latest-small/movies.csv  


**About the Data**

In [8]:
!cat ../../data/recommendations/ml-latest-small/README.txt

Summary

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for down

For this model, we will be using `ratings.csv` mainly, which contains 4 columns,
- userId
- movieId
- rating
- timestamp

## 2. Read data and perform train and test split

In [9]:
import os
import numpy as np
import pandas as pd

In [10]:
# read rating data
DATA_DIR = '../../data/recommendations/'
fpath = 'ml-latest-small/ratings.csv'
df = pd.read_csv(os.path.join(DATA_DIR, fpath))

In [11]:
# let's see what the data look like
df.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [12]:
# understand what's the maximum number of hold out portion should be
df.groupby('userId').movieId.nunique().min()

20

Note: Since the "least active" user has 20 ratings, for our testing set, let's hold out 10 items for every user so that the max test set portion is 50%.

In [15]:
def train_test_split(df, holdout_num):
    """ perform training/testing split
    
    @param df: dataframe
    @param holdhout_num: number of items to be held out
    
    @return df_train: training data
    @return df_test testing data
    
    """
    # first sort the data by time
    df = df.sort_values(['userId', 'timestamp'], ascending=[True, False])
    
    # perform deep copy on the dataframe to avoid modification on the original dataframe
    df_train = df.copy(deep=True)
    df_test = df.copy(deep=True)
    
    # get test set
    df_test = df_test.groupby(['userId']).head(holdout_num).reset_index()
    
    # get train set
    df_train = df_train.merge(
        df_test[['userId', 'movieId']].assign(remove=1),
        how='left'
    ).query('remove != 1').drop('remove', axis=1).reset_index(drop=True)
    
    # sanity check to make sure we're not duplicating/losing data
    assert len(df) == len(df_train) + len(df_test)
    
    return df_train, df_test

In [16]:
df_train, df_test = train_test_split(df, 10)

## 3. Perform negative sampling

Assuming if a user rating an item is a positive label, there is no negative sample in the dataset, which is not possible for model training. Therefore, we random sample `n` items from the unseen movie list for every user to provide the negative samples.

In [17]:
def negative_sampling(user_ids, movie_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label
    
    @param user_ids: list of user ids
    @param movie_ids: list of movie ids
    @param items: unique list of movie ids
    @param n_neg: number of negative labels to sample
    
    @return df_neg: negative sample dataframe
    
    """
    
    neg = []
    ui_pairs = zip(user_ids, movie_ids)
    records = set(ui_pairs)
    
    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            # if the randomly sampled movie exists for that user
            j = np.random.choice(items)
            while(u, j) in records:
                # resample
                j = np.random.choice(items)
            neg.append([u, j, 0])
    # conver to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['userId', 'movieId', 'rating'])
    
    return df_neg

In [22]:
# create negative samples for training set
neg_train = negative_sampling(
    user_ids=df_train.userId.values, 
    movie_ids=df_train.movieId.values,
    items=df.movieId.unique(),
    n_neg=5
)
neg_train.head(2)

,userId,movieId,rating
0,247,111360,0
1,247,4920,0


In [23]:
print(f'created {neg_train.shape[0]:,} negative samples')

created 2,498,660 negative samples


In [24]:
df_train = df_train[['userId', 'movieId']].assign(rating=1)
df_test = df_test[['userId', 'movieId']].assign(rating=1)

df_train = pd.concat([df_train, neg_train], ignore_index=True)

df_train.head(2)

,userId,movieId,rating
0,1,151,1
1,1,1090,1


## 4. Calulate statistics for our understanding and model training

In [25]:
def get_unique_count(df):
    """calculate unique user and movie counts"""
    return df.userId.nunique(), df.movieId.nunique()

In [26]:
# unique number of user and movie in the whole dataset
get_unique_count(df)

(610, 9724)

In [27]:
print('training set shape', get_unique_count(df_train))
print('testing set shape', get_unique_count(df_test))

training set shape (610, 9724)
testing set shape (610, 2583)


Next, we calculate some statistics for training purpose.

In [28]:
# number of unique user and number of unique item/movie
n_user, n_item = get_unique_count(df_train)

print("number of unique users", n_user)
print("number of unique items", n_item)

number of unique users 610
number of unique items 9724


In [29]:
# save the variable for the model training notebook
# -----
# read about `store` magic here: 
# https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html

%store n_user
%store n_item

Stored 'n_user' (int)
Stored 'n_item' (int)


## 5. Save data

In [30]:
dest = '../../data/recommendations/movie-npy'
train_path = os.path.join(dest, 'train.npy')
test_path = os.path.join(dest, 'test.npy')

!mkdir {dest}
np.save(train_path, df_train.values)
np.save(test_path, df_test.values)